### Preamble

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 6. Higher-order functions and modular programming

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [25]:
def recursionTemplate[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case (h: A) :: (t: List[A]) =>  
            val tailSol: B = recursionTemplate(t)
            ???(h: A, tailSol: B) : B

defined function recursionTemplate

In [26]:
def recursionTemplate[A, B](l: List[A])(
        nil: B, 
        cons: (A, B) => B
    ): B = 
    l match 
        case Nil => nil : B
        case (h: A) :: (t: List[A]) =>  
            val tailSol: B = recursionTemplate(t)(nil, cons)
            cons(h, tailSol) : B

defined function recursionTemplate

In [2]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case h :: (t: List[Int]) =>  
            val tailSum: Int = sum(t)
            h + tailSum : Int

defined function sum

In [27]:
def sum(l: List[Int]): Int = 
    // Utiliza el esquema de recursión para tipos A =Int y B=Int
    // devolviendo 0 cuando la lista sea vacía
    // y en caso de que la lista no sea vacía, y tenga cabeza `h`
    // y solución al resto `tailSol`, devuelve h + tailSol
    recursionTemplate[Int, Int](l)(
        0, // solución si la lista es vacía)
        (head: Int, tailSol: Int) => head + tailSol // solución si la lista es no vacía, con cabeza `head` y solución al resto `tailSol`
    )

defined function sum

In [47]:
def sum(l: List[Int]): Int = 
    l.foldRight[Int](
        0
    )(
        (head: Int, tailSol: Int) => head + tailSol // solución si la lista es no vacía, con cabeza `head` y solución al resto `tailSol`
    )

defined function sum

In [48]:
def sum(l: List[Int]): Int = 
    l.foldRight(0)((head: Int, tailSol: Int) => head + tailSol)

defined function sum

In [49]:
def sum(l: List[Int]): Int = 
    l.foldRight(0)((head, tailSol) => head + tailSol)

defined function sum

In [50]:
def sum(l: List[Int]): Int = 
    l.foldRight(0)(_+_)

defined function sum

In [28]:
sum(List(1,2,3,4))

res28: Int = 10

In [37]:
def evenOdd[A](l: List[A], even: Boolean): List[A] =
    l match 
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailSol: List[A] = evenOdd(tail, !even)
            if even then (head :: tailSol) : List[A]
            else tailSol

defined function evenOdd

In [ ]:
val f1: (Int, String) => Boolean = (_, _) => false
val f2: Int => (String => Boolean)

In [37]:
def evenOdd[A](l: List[A], even: Boolean): List[A] =
    l match 
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailSol: List[A] = evenOdd(tail, !even)
            if even then (head :: tailSol) : List[A]
            else tailSol

defined function evenOdd

In [39]:
def evenOdd[A](l: List[A]): Boolean => List[A] =
    l match 
        case Nil => ((b: Boolean) => Nil) : (Boolean => List[A])
        case head :: tail => 
            val tailSol: Boolean => List[A] = evenOdd(tail)
            even => 
                if even then (head :: tailSol(!even)) : List[A]
                else tailSol(!even)

defined function evenOdd

In [42]:
def evenOdd[A](l: List[A]): Boolean => List[A] =
    recursionTemplate[A, Boolean => List[A]](l)(
        ((b: Boolean) => Nil) : (Boolean => List[A]),
        (head: A, tailSol: Boolean => List[A]) => 
            even => 
                if even then (head :: tailSol(!even)) : List[A]
                else tailSol(!even)
    )

defined function evenOdd

In [44]:
object Std: 

    def recursionTemplate[A, B](l: List[A])(nil: B, cons: (A, B) => B): B = 
        ??? 

    class List[A]: 
        def foldRight[B](nil: B)(cons: (A, B) => B): B = ??? 

defined object Std

In [45]:
def evenOdd[A](l: List[A]): Boolean => List[A] =
    l.foldRight[Boolean => List[A]](
        ((b: Boolean) => Nil) : (Boolean => List[A])
    )(
        (head: A, tailSol: Boolean => List[A]) => 
            even => 
                if even then (head :: tailSol(!even)) : List[A]
                else tailSol(!even)
    )

defined function evenOdd

In [37]:
def evenOdd[A](l: List[A], even: Boolean): List[A] =
    recursionTemplate[???, ???](l)(
        ???, // solución si lista vacía
        (head, tailSol) => ??? // ...
    )

defined function evenOdd

In [41]:
evenOdd(List(0,1,2,3,4,5,6))(false)

res41: List[Int] = List(1, 3, 5)

In [33]:
evenOdd(List(0,1,2,3,4,5,6), true) == List(0,2,4,6)
evenOdd(7 :: List(2,2,8,3,4,7), true) == {
    val tailSol = List(2,3,7)
    7 :: List(2,3,7)
}
evenOdd(List(), true) == List()

res33_0: Boolean = false
res33_1: Boolean = false
res33_2: Boolean = true

In [38]:
evenOdd(List(0,1,2,3,4,5,6), false) == List(1, 3, 5)
evenOdd(7 :: List(2,2,8,3,4,7), false) == {
    
    List(2, 8, 4)
}

evenOdd(List(), false) == List()


res38_0: Boolean = true
res38_1: Boolean = true
res38_2: Boolean = true

In [ ]:
def evens[A](l: List[

In [13]:
def firstLetters(l: List[String]): String = 
    l match 
        case Nil => "" : String
        case (h: String) :: (t: List[String]) =>  
            val tailSol: String = firstLetters(t)
            h(0).toString + tailSol : String

defined function firstLetters

In [13]:
def firstLetters(l: List[String]): Either[String, Unit] = 
    l match 
        case Nil => "" : String
        case (h: String) :: (t: List[String]) =>  
            val tailSol: String = firstLetters(t)
            h(0).toString + tailSol : String

defined function firstLetters

In [18]:
val o: Option[String] = Some("hola")
val o1: Option[String] = None

o: Option[String] = Some(value = "hola")
o1: Option[String] = None

In [19]:
o.get

res19: String = "hola"

In [20]:
o1.get

java.util.NoSuchElementException: None.get

In [21]:
def firstLetters(l: List[String]): Option[String] = 
    l match 
        case Nil => Some("") : Option[String]
        case (h: String) :: (t: List[String]) =>  
            val tailSol: Option[String] = firstLetters(t)
            // h(0).toString + tailSol : Option[String]
            if tailSol == None || h == "" then 
                None
            else 
                Some(h(0).toString + tailSol.get) : Option[String]

defined function firstLetters

In [24]:
def firstLetters(l: List[String]): Option[String] = 
    l match 
        case Nil => Some("") : Option[String]
        case (h: String) :: (t: List[String]) =>  
            val tailSol: Option[String] = firstLetters(t)
            tailSol match 
                case Some(ts) if h != "" => 
                    Some(h(0).toString + ts) : Option[String]
                case _ => 
                    None

defined function firstLetters

In [ ]:
def firstLetters(l: List[String]): Option[String] = 
    recursionTemplate[String, Option[String]](l)(
        ??? : Option[String], 
        (head: ???, tailSol: ???) => ??? : Option[String]
    )

In [29]:
def firstLetters(l: List[String]): Option[String] = 
    recursionTemplate[String, Option[String]](l)(
        Some("") : Option[String], 
        (head: String, tailSol: Option[String]) => 
            tailSol match 
                case Some(ts) if head != "" => 
                    Some(head(0).toString + ts) : Option[String]
                case _ => 
                    None
    )

defined function firstLetters

In [46]:
def firstLetters(l: List[String]): Option[String] = 
    l.foldRight[Option[String]](
        Some("") : Option[String]
    )(
        (head: String, tailSol: Option[String]) => 
            tailSol match 
                case Some(ts) if head != "" => 
                    Some(head(0).toString + ts) : Option[String]
                case _ => 
                    None
    )

defined function firstLetters

In [30]:
firstLetters(List("hola", "adios", "bye")) == Some("hab")

res30: Boolean = true

In [23]:
firstLetters("hola" :: List("", "bye")) == None

res23: Boolean = true

In [17]:
firstLetters(List("hola", "", "bye")) == "hab"

java.lang.StringIndexOutOfBoundsException: String index out of range: 0

In [10]:
"hola".apply(0)

res10: Char = 'h'

In [16]:
"".apply(0)

java.lang.StringIndexOutOfBoundsException: String index out of range: 0

In [12]:
'h'.toString + "ola"

res12: String = "hola"

In [15]:
firstLetters("hola" :: List("adios", "bye")) == {
    val tailSol: String = "ab"
    // "hab"
    "hola"(0).toString + tailSol
}

res15: Boolean = true

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [4]:
// monomorphic `combine` method

defined function combine

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def recursionTemplate[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case head :: tail => 
            val tailSol: B = recursionTemplate(tail) 
            ???(head, tailSol)

In [3]:
multiply(List(7,2,3,4)) == 7*2*3*4
multiply(List(4)) == 4
multiply(List()) == 1

res3_0: Boolean = true
res3_1: Boolean = true
res3_2: Boolean = true

In [4]:
multiply(7 :: List(2,3,4)) == {
    val head: Int = 7
    val tailSol: Int = 2*3*4
    // 7 * 2 * 3 * 4
    head * tailSol
}

res4: Boolean = true

In [14]:
List(1,2,3) ++ (0 :: List.fill(10000)(1))

res14: List[Int] = List(
  1,
  2,
  3,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
...

In [15]:
def multiply(l: List[Int]): Int = 
    l match 
        case Nil => 1 : Int
        case 0 :: tail => 0
        case head :: tail => 
            val tailSol: Int = multiply(tail) 
            head * tailSol

defined function multiply

In [2]:
def multiply(l: List[Int]): Int = 
    l match 
        case Nil => 1 : Int
        case head :: tail => 
            val tailSol: Int = multiply(tail) 
            head * tailSol

defined function multiply

In [ ]:
def recursionTemplate[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case head :: tail => 
            val tailSol: B = recursionTemplate(tail) 
            ???(head, tailSol)

In [5]:
def recursionTemplate[A, B](l: List[A])(
        nil: B, cons: (A, B) => B): B = 
    l match 
        case Nil => nil : B
        case head :: tail => 
            val tailSol: B = recursionTemplate(tail)(nil, cons)
            cons(head, tailSol): B

defined function recursionTemplate

In [6]:
def multiply(l: List[Int]): Int = 
    recursionTemplate(l)(
        1 : Int,
        (head, tailSol) => 
            head * tailSol
    )

defined function multiply

In [7]:
def multiply(l: List[Int]): Int = 
    l.foldRight(
        1 : Int
    )(
        (head, tailSol) => 
            head * tailSol
    )

defined function multiply

In [8]:
def multiply(l: List[Int]): Int = 
    l.foldRight(1)((head, tailSol) => head * tailSol)

defined function multiply

In [10]:
def multiply(l: List[Int]): Int = 
    l.foldRight(1)(_*_)

defined function multiply

In [11]:
def sum(l: List[Int]): Int = 
    l.foldRight(0)(_+_)

defined function sum

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
// more generically: foldRight


Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [6]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // direct solution to atomic problem
        (head, subsol) => head + subsol // composing sub-solution
    )

defined function sum

In [7]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // direct solution
        (head: Int, subsol: Int) => head * subsol // compose
    )

defined function multiply

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [9]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

defined object Standard

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

In [10]:
// foldRight over lists



res10: Int = 6

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [11]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [12]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

In [13]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (unfortunately, the concept of catamorphism won't be explained due to lack of time). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

out           aux
---           --- 
???           List(1,2,3)
???(1, out)   List(2,3)
???(2, out)   List(3)
???(3, out)   List()

out           aux
---           --- 
0             List(1,2,3)
1+0           List(2,3)
1+(1+0)       List(3)
1+(1+1+0)     List()

In [24]:
def iterativeTemplate_Imperatively[A, B](l: List[A]): B = 
    var out: B = ??? 
    var aux: List[A] = l
    while aux != Nil do 
        out = (??? : (A, B) => B)(l.head, out)
        aux = aux.tail
    out

defined function iterativeTemplate_Imperatively

In [30]:
def length_Imperatively[A](l: List[A]): Int = 
    println("out" +  "\t\t\t" + "aux")
    println("---" +  "\t\t\t" + "---")
    var out: Int = 0 
    var aux: List[A] = l
    while aux != Nil do 
        println(out.toString + "\t\t\t" + aux)
        out = 1+out // (??? : (A, Int) => Int)(l.head, out)
        aux = aux.tail
    out

defined function length_Imperatively

In [31]:
length_Imperatively(List(1,2,3,4))

out			aux
---			---
0			List(1, 2, 3, 4)
1			List(2, 3, 4)
2			List(3, 4)
3			List(4)


res31: Int = 4

In [49]:
def iterativeTemplate_TailRecursion[A, B](l: List[A]): B = 
    
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                val out1 = (??? : (B, A) => B)(out, l.head)
                val aux1 = aux.tail
                step(out, aux)

    step(??? : B, l)

defined function iterativeTemplate_TailRecursion

In [50]:
def iterativeTemplate_TailRecursion[A, B](l: List[A]): B = 
    
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                step((??? : (B, A) => B)(out, e), tail)

    step(??? : B, l)

defined function iterativeTemplate_TailRecursion

In [33]:
def length_TailRecursion[A](l: List[A]): Int = 
    
    def step(out: Int, aux: List[A]): Int = 
        println(out.toString + "\t\t\t" + aux)
        aux match 
            case Nil => out
            case e :: tail => 
                step(1+out, tail)

    println("out" +  "\t\t\t" + "aux")
    println("---" +  "\t\t\t" + "---")
    
    step(0 : Int, l)

defined function length_TailRecursion

In [34]:
length_TailRecursion(List(1,2,3,4))

out			aux
---			---
0			List(1, 2, 3, 4)
1			List(2, 3, 4)
2			List(3, 4)
3			List(4)
4			List()


res34: Int = 4

In [ ]:
out: List[A]         aux: List[A]
------------         ------------
Nil                  List(1,2,3)
1 :: Nil             List(2,3)
2 :: (1 :: Nil)      List(3)
3 :: (2 :: 1 :: Nil) List()

In [37]:
def reverse_TailRecursion[A](l: List[A]): List[A] = 
    
    def step(out: List[A], aux: List[A]): List[A] = 
        println(out.toString + "\t\t\t" + aux)
        aux match 
            case Nil => out
            case e :: tail => 
                step(e :: out, tail)

    println("out" +  "\t\t\t" + "aux")
    println("---" +  "\t\t\t" + "---")
    
    step(Nil : List[A], l)

defined function reverse_TailRecursion

In [38]:
reverse_TailRecursion(List(1,2,3,4,5))

out			aux
---			---
List()			List(1, 2, 3, 4, 5)
List(1)			List(2, 3, 4, 5)
List(2, 1)			List(3, 4, 5)
List(3, 2, 1)			List(4, 5)
List(4, 3, 2, 1)			List(5)
List(5, 4, 3, 2, 1)			List()


res38: List[Int] = List(5, 4, 3, 2, 1)

In [35]:
List(1,2,3,4).foldRight(Nil: List[Int])(
    (head, tailReversed) => tailReversed :+ head)

res35: List[Int] = List(4, 3, 2, 1)

In [15]:
// reverse function, imperatively


defined function reverse

In [51]:
def iterativeTemplate_TailRecursion[A, B](l: List[A]): B = 
    
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                step((??? : (B, A) => B)(out, e), tail)

    step(??? : B, l)

defined function iterativeTemplate_TailRecursion

In [52]:
def reverse[A](l: List[A]): List[A] = 
    
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case e :: tail => 
                step(e :: out, tail)

    step(Nil, l)

defined function reverse

In [53]:
def iterativeTemplate_TailRecursion[A, B](l: List[A])(
        initial: B, update: (B, A) => B): B = 
    @annotation.tailrec
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                step(update(out, e), tail)

    step(initial : B, l)

defined function iterativeTemplate_TailRecursion

In [54]:
def reverse[A](l: List[A]): List[A] = 
    iterativeTemplate_TailRecursion[A, List[A]](l)(
        Nil, (out, e) => e :: out
    )

defined function reverse

In [44]:
def reverse[A](l: List[A]): List[A] = 
    l.foldLeft(
        Nil : List[A]
    )((out, e) => e :: out
    )

defined function reverse

In [45]:
def reverse[A](l: List[A]): List[A] = 
    l.foldLeft[List[A]](Nil)((out, e) => e :: out)

defined function reverse

In [43]:
reverse(List(1,2,3,4,5))

res43: List[Int] = List(5, 4, 3, 2, 1)

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [16]:
// abstraction: foldLeft, imperatively


defined function foldLeft

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [48]:
@annotation.tailrec
def iterativeTemplate_TailRecursion[A, B](l: List[A])(
        initial: B, update: (A, B) => B): B = 
    l match 
        case Nil => initial
        case e :: tail => 
            iterativeTemplate_TailRecursion(tail)(update(e, initial), update)

defined function iterativeTemplate_TailRecursion

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ??? 

-- [E097] Syntax Error: cell1.sc:2:4 -------------------------------------------
2 |def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
  |    ^
  |TailRec optimisation not applicable, method foldLeft contains no recursive calls
Compilation Failed

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [18]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [19]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [20]:
def reverse[A](list: List[A]): List[A] = 
    ??? 

defined function reverse

But we may also use the `foldRight` function to implement them:

In [21]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [22]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [24]:
def reverse[A](list: List[A]): List[A] = 
    ???

defined function reverse

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [26]:
// blowing up the stack 


java.lang.StackOverflowError: null

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [1]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

defined function foldRightTR

This implementation works right with big lists:

In [28]:
// check it out


res28: Int = 0

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [29]:
// foldLeftAux


defined function foldLeftAux

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [30]:
// foldLeftAux with foldRight


defined function foldLeftAux

The original signature can then be implemented as follows: 

In [2]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

## The `map` HOF

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../images/map.svg)

The implementation for lists can be done as follows:

In [5]:
120.toChar
123.toChar
124.toChar

res5_0: Char = 'x'
res5_1: Char = '{'
res5_2: Char = '|'

In [6]:
// map example 1
def chars(l: List[Int]): List[Char] = 
    l match 
        case Nil => ??? : List[Char]
        case head :: tail => 
            val tailSol: List[Char] = ??? 
            ???(head, tailSol): List[Char]

defined function chars

In [8]:
// map example 1
def chars(l: List[Int]): List[Char] = 
    l match 
        case Nil => Nil : List[Char]
        case head :: tail => 
            val tailSol: List[Char] = chars(tail)
            head.toChar :: tailSol : List[Char]

defined function chars

In [11]:
chars(List(120,123,124)) == List('x', '{', '|')
chars(120 :: List(123,124)) == 'x' :: List('{', '|')                                        

res11_0: Boolean = true
res11_1: Boolean = true

In [15]:
// map example 2
def empties(l: List[String]): List[Boolean] = 
    l match 
        case Nil => Nil : List[Boolean]
        case head :: tail => 
            val tailSol: List[Boolean] = empties(tail)
            (head == "") :: tailSol : List[Boolean]

defined function empties

In [16]:
empties("abc" :: List("", "hola", "", "adios")) == {
    val head = "abc"
    val tail = List("", "hola", "", "adios")
    val tailSol = List(true, false, true, false)
    // List(false, true, false, true, false)
    (head == "") :: List(true, false, true, false)
}

res16: Boolean = true

In [8]:
// map example 1
def map(l: List[???]): List[???] = 
    l match 
        case Nil => Nil : List[???]
        case head :: tail => 
            val tailSol: List[???] = map(tail)
            ???(head) :: tailSol : List[???]

defined function chars

In [8]:
// map example 1
def smash(l: List[Tomate]): List[Zumo] = 
    l match 
        case Nil => Nil : List[Zumo]
        case head :: tail => 
            val tailSol: List[Zumo] = map(tail)
            aplasta(head) :: tailSol : List[Zumo]

defined function chars

In [24]:
object Std: 

    abstract class List[A]: 
        def map[B](f: A => B): List[B]

defined object Std

In [17]:
// map example 1
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match 
        case Nil => Nil : List[B]
        case (head: A) :: (tail: List[A]) => 
            val tailSol: List[B] = map(tail)(f)
            (f(head) :: tailSol) : List[B]

defined function map

In [26]:
// map example 2
def empties(l: List[String]): List[Boolean] = 
    map[String, Boolean](l)(head => head == "")

defined function empties

In [27]:
// map example 2
def empties(l: List[String]): List[Boolean] = 
    l.map[Boolean](head => head == "")

defined function empties

In [19]:
// map example 2
def empties(l: List[String]): List[Boolean] = 
    map(l)(_ == "")

defined function empties

In [20]:
// map example 1
def chars(l: List[Int]): List[Char] = 
    map(l)(head => head.toChar)

defined function chars

In [21]:
// map example 1
def chars(l: List[Int]): List[Char] = 
    map(l)(_.toChar)

defined function chars

In [23]:
// map example 1
def chars(l: List[Int]): List[Char] = 
    l.map(_.toChar)

defined function chars

In [2]:
// map generically


The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [22]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends AnyFlatSpec with should.Matchers:

    val isEven: Int => Boolean = _ % 2 == 0

    "mapping an empty list" should "return an empty list" in:
        map(List())(isEven) shouldBe ???

    "mapping an non-empty list" should "only change its content" in:
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [34]:
run(TestMap(map[Int, Boolean]))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


although it's not necessary to write the types explicitly (Scala will infer the types for us):

In [35]:
run(TestMap(map))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [17]:
// map example 1
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match 
        case Nil => Nil : List[B]
        case (head: A) :: (tail: List[A]) => 
            val tailSol: List[B] = map(tail)(f)
            (f(head) :: tailSol) : List[B]

defined function map

In [17]:
// map example 1
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l.foldRight(??? : List[B])(
        (head, tailSol) => ??? : List[B]
    )

defined function map

In [28]:
// map example 1
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l.foldRight(Nil : List[B])(
        (head, tailSol) => f(head) :: tailSol : List[B]
    )

defined function map

In [36]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

defined function mapFR

In [37]:
run(TestMap(mapFR))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [38]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "mapping the `None` value" should "return `None`" in:
        map(None)(isEven) shouldBe ???
    
    "mapping a `Some` value" should "only change its content" in:
        map(Option(3))(isEven) shouldBe ???
        map(Option(1))(isEven) shouldBe ???
        map(Option(2))(isEven) shouldBe ???

defined class TestMapOption

The implementation is simple:

In [39]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

defined function mapOpt

In [40]:
run(TestMapOption(mapOpt))

cell38$Helper$TestMapOption:
mapping the `None` value
- should return `None`
mapping a `Some` value
- should only change its content


## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../images/filter.svg)

In [ ]:
def even(l: List[Int]): List[Int] = 
    l match 
        case Nil => ??? : List[Int]
        case head :: tail => 
            val tailSol: List[Int] = even(tail)
            ???(head, tailSol)

In [29]:
def even(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case head :: tail => 
            val tailSol: List[Int] = even(tail)
            if head % 2 == 0 then head :: tailSol
            else tailSol

defined function even

In [29]:
def even(l: List[Int]): List[Int] = 
    filter(l)(head => head % 2 == 0)

defined function even

In [33]:
def even(l: List[Int]): List[Int] = 
    l.filter(head => head % 2 == 0)

defined function even

In [37]:
def even(l: List[Int]): List[Int] = 
    l.filter(_ % 2 == 0)

defined function even

In [34]:
even(List(1,2,3,4,5)) == List(2,4)
even(List(1,3,5)) == List()

res34_0: Boolean = true
res34_1: Boolean = true

In [31]:
def nonempties(l: List[String]): List[String] = 
    l match 
        case Nil => Nil : List[String]
        case head :: tail => 
            val tailSol: List[String] = nonempties(tail)
            if head != "" then head :: tailSol
            else tailSol

defined function nonempties

In [31]:
def nonempties(l: List[String]): List[String] = 
    filter(l)(head => head != "")

defined function nonempties

In [35]:
def nonempties(l: List[String]): List[String] = 
    l.filter(head => head != "")

defined function nonempties

In [36]:
def nonempties(l: List[String]): List[String] = 
    l.filter(_ != "")

defined function nonempties

In [ ]:
def maduros(l: List[Tomate]): List[Tomate] = 
    l match 
        case Nil => Nil : List[Tomate]
        case head :: tail => 
            val tailSol: List[Tomate] = nonempties(tail)
            if esMaduro(head) then head :: tailSol
            else tailSol

In [ ]:
def maduros(l: List[Tomate]): List[Tomate] = 
    filter(l)(esMaduro) 
    //filter(l)( (t: Tomate) => esMaduro(t)) 

In [38]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l match 
        case Nil => Nil : List[A]
        case head :: tail => 
            val tailSol: List[A] = filter(tail)(pred)
            if pred(head) then head :: tailSol
            else tailSol

defined function filter

In [43]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldRight(Nil : List[A]):
        (head, tailSol) => 
            if pred(head) then head :: tailSol
            else tailSol

defined function filter

out                  aux
---                  --- 
Nil                  List(1,2,4,5,6)
Nil                  List(2,4,5,6)
List(2)              List(4,5,6)
List(2,4)            List(5,6)
List(2,4)            List(6)
List(2,4,6)          List()

In [45]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldLeft(Nil : List[A]): 
        (out: List[A], a: A) =>
            if pred(a) then out :+ a : List[A]
            else out

defined function filter

In [53]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldLeft{
        println("out\t\t\t\taux\n---\t\t\t\t---")
        Nil : List[A]
    }{
        (out: List[A], a: A) =>
            println(out.toString + "\t\t\t\t" + a.toString)
            if pred(a) then a :: out : List[A]
            else out
    }.reverse

defined function filter

In [54]:
filter(List(1,2,3,4,5,6,8))(_ % 2 == 0)

out				aux
---				---
List()				1
List()				2
List(2)				3
List(2)				4
List(4, 2)				5
List(4, 2)				6
List(6, 4, 2)				8


res54: List[Int] = List(2, 4, 6, 8)

In [44]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldLeft(??? : List[A]): 
        (out: List[A], a: A) =>
            ??? : List[A]

defined function filter

In [41]:
filter(List(2,4,6))(_ % 2 == 0)
List(1,2,3,4,5,6).filter(_ % 2 != 0)
List("a", "", "b", "c", "d").filter(_ != "")

res41_0: List[Int] = List(2, 4, 6)
res41_1: List[Int] = List(1, 3, 5)
res41_2: List[String] = List("a", "b", "c", "d")

In [ ]:
def filter(l: List[???]): List[???] = 
    l match 
        case Nil => Nil : List[???]
        case head :: tail => 
            val tailSol: List[???] = filter(tail)
            if ???(head) then head :: tailSol
            else tailSol

In [32]:
nonempties(List("abc", "", "hola", "adios", "")) == List("abc", "hola", "adios")

res32: Boolean = true

In [41]:
// filter, generically 

defined function filterR

In [42]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(List())(isEven) shouldBe ???
        filter(List(1))(isEven) shouldBe ???
        filter(List(1,3,5))(isEven) shouldBe ???
        filter(List(2,4,6))(isEven) shouldBe ???

defined class TestFilterList

In [43]:
run(TestFilterList(filterR))

cell42$Helper$TestFilterList:
filter
- should work


Using `foldRight` we can get a more modular implementation: 

In [45]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

defined function filterFR

In [46]:
run(TestFilterList(filterFR))

cell42$Helper$TestFilterList:
filter
- should work


### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [48]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(None)(isEven) shouldBe ???
        filter(Some(1))(isEven) shouldBe ???
        filter(Some(2))(isEven) shouldBe ???

defined class TestFilterOption

In [49]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

defined function filter

In [50]:
run(TestFilterOption(filter))

cell48$Helper$TestFilterOption:
filter
- should work


## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [52]:
class TestWords(
    words: List[String] => List[String]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in:
        words(paragraph1) shouldBe 
            ???

defined class TestWords

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [53]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")

???

paragraph: List[String] = List("en un lugar", "de la Mancha")
res53_1: List[List[String]] = List(
  List("en", "un", "lugar"),
  List("de", "la", "Mancha")
)

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [54]:
// flatten


defined function flatten

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [55]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

In [57]:
run(TestWords(words))

cell52$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [58]:
// flatMap 


defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [59]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

### FlatMapping optional values

In [60]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

defined function flatMap

In [61]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in:
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???

defined class TestFlatMapOption

In [62]:
run(TestFlatMapOption(flatMap))

cell61$Helper$TestFlatMapOption:
flatMap
- should work


## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [64]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in:
        lengths(paragraph1) shouldBe 
            List(2, 2, 5, 2, 2, 6, 
                 2, 4, 6, 2, 6, 9)

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [70]:
def lengthsM(paragraph: List[String]): List[Int] =
    ???

defined function lengthsM

and this works, of course: 

In [69]:
run(TestLengths(lengthsM))

cell64$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [71]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsHOF

In [73]:
run(TestLengths(lengthsHOF))

cell64$Helper$TestLengths:
lengths
- should work


This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [3]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsFC

In [77]:
run(TestLengths(lengthsFC))

cell64$Helper$TestLengths:
lengths
- should work
